In [1]:
# Imports
import numpy as np
import lmdb
import caffe
import PIL.Image
from StringIO import StringIO

# Set paths (Currently for GoogLeNet-384x288c244-finetuning)
MODEL_PATH    = '/home/reneoctavio/Downloads/digits-2.0-final/digits/digits/jobs/20160128-193319-9f2b/'
DATASET_PATH  = '/home/reneoctavio/Downloads/digits-2.0-final/digits/digits/jobs/20160128-180614-0b93/'
SNAPSHOT_PATH = '/home/reneoctavio/Downloads/digits-2.0-final/digits/digits/jobs/20160128-193319-9f2b/'
OUTPUT_PATH   = '/home/reneoctavio/Documents/Plantas/Caffe/GoogLeNet-384x288c244-finetuning/'

# Input Files
MODEL_FILE    = MODEL_PATH    + 'deploy.prototxt'
PRETRAINED    = SNAPSHOT_PATH + 'snapshot_iter_9090.caffemodel'
LMDB_PATH     = DATASET_PATH  + 'test_db'
LABEL_FILE    = DATASET_PATH  + 'labels.txt'
MEAN_FILE     = OUTPUT_PATH   + 'imagenet_mean.npy'

# Output Files
PRED_TOP5_FILE  = OUTPUT_PATH + 'prediction-top-5-test.npy'
PRED_SCORE_FILE = OUTPUT_PATH + 'prediction-score-test.npy'
TRUE_LABEL_FILE = OUTPUT_PATH + 'true-label-test.npy'

In [2]:
# Read Labels
label_file = open(LABEL_FILE, 'r')
label_dic = {}
idx = 0
for line in label_file:
    line = line.rsplit('\n')
    label_dic[idx] = line
    idx = idx + 1

In [3]:
# Read network
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)

In [4]:
# Input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.load(MEAN_FILE).mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2, 1, 0))  # the reference model has channels in BGR order instead of RGB

In [5]:
# Read images path and label
lmdb_env = lmdb.open(LMDB_PATH)  # equivalent to mdb_env_open()
lmdb_txn = lmdb_env.begin()  # equivalent to mdb_txn_begin()
lmdb_cursor = lmdb_txn.cursor()  # equivalent to mdb_cursor_open()

In [9]:
# Classify images
true_label = []
pred_top_5 = []
pred_score = []

for key, value in lmdb_cursor:
    # Read Datum
    datum = caffe.proto.caffe_pb2.Datum()
    datum.ParseFromString(value)
    
    # Get image label 
    label = int(datum.label)
    true_label.append(label)
    
    # Image is uint8 [0-255] -> transform to -> float32 [0-1]
    s = StringIO()
    s.write(datum.data)
    s.seek(0)
    img = PIL.Image.open(s)
    image = np.array(img)
    image = (image.astype(np.float32)) / 255.0
    
    # Sample 10 crops and reflections
    input_oversampled = caffe.io.oversample([image], (224, 224))
        
    # 'data' is the input blob name in the model definition, so we preprocess for that input.
    caffe_input = np.asarray([transformer.preprocess('data', in_) for in_ in input_oversampled])
        
    # set net to batch size of 10
    net.blobs['data'].reshape(10, 3, 224, 224)

    # Pass image forward and predict
    net.blobs['data'].data[...] = caffe_input
    out = net.forward()
    
    # Average predictions and give 5 best
    predictions = out['prob']
    predictions = predictions.reshape((len(predictions) / 10, 10, -1))
    predictions = predictions.mean(1)
    predictions = predictions[0]
    
    # Save all classes score for each image
    pred_score.append(predictions)
    
    # Save Top-5 predictions
    predictions = predictions.argsort()[-5:][::-1]
    pred_top_5.append(predictions)

    print("Image " + key)
    print("True Class: " + label_dic[label][0])
    print("Pred Class: " + label_dic[predictions[0]][0])
    print("\n")

Image 00000000_21
True Class: Rhododendron simsii
Pred Class: Bougainvillea glabra


Image 00000001_47
True Class: Eugenia sprengelii
Pred Class: Eugenia sprengelii


Image 00000002_39
True Class: Phoenix roebelenii
Pred Class: Phoenix roebelenii


Image 00000003_31
True Class: Agave ovatifolia
Pred Class: Agave ovatifolia


Image 00000004_14
True Class: Agave attenuata
Pred Class: Agave attenuata


Image 00000005_7
True Class: Buxus microphylla
Pred Class: Buxus microphylla


Image 00000006_27
True Class: Strelitzia reginae
Pred Class: Strelitzia reginae


Image 00000007_16
True Class: Begonia X hybrida
Pred Class: Begonia X hybrida


Image 00000008_21
True Class: Rhododendron simsii
Pred Class: Rhododendron simsii


Image 00000009_33
True Class: Hibiscus rosa-sinensis
Pred Class: Hibiscus rosa-sinensis


Image 00000010_14
True Class: Agave attenuata
Pred Class: Agave attenuata


Image 00000011_46
True Class: Clerodendrum X speciosum
Pred Class: Clerodendrum X speciosum


Image 000000

In [10]:
# Save List
np.save(PRED_TOP5_FILE, np.array(pred_top_5))
np.save(PRED_SCORE_FILE, np.array(pred_score))
np.save(TRUE_LABEL_FILE, np.array(true_label))